In [17]:
# Import necessary packages: 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import scipy.linalg as linalg

## Finite Difference option pricing: 

In [2]:
# Define the Finite Difference Class: 

class FiniteDifferences: 
  
  # Initiate the object:
  def __init__(self, S0, K, r, T, sigma, Smax, M, N, callable = True):
    # Define parameters for the object: 
    self.S0 = S0
    self.K = K
    self.r = r
    self.T = T
    self.sigma = sigma
    self.Smax = Smax
    self.M = int(M)
    self.N = int(N)
    self.callable = callable

    # Caclulate the parameters: 
    self.dS = Smax/float(self.M) 
    self.dt = T/float(self.N)
    self.i_values = np.arange(self.M)
    self.j_values = np.arange(self.N)
    self.grid = np.zeros(shape=(self.M+1, self.N + 1))
    self.boundary_conds = np.linspace(0, Smax, self.M+1)

  def _setup_boundary_conditions_(self):
    pass


  def _setup_coefficients_(self):
    pass

  def _traverse_grid_(self):
    pass

  def _interpolate_(self):

    return np.interp(self.S0, self.boundary_conds, self.grid[:,0])  

  def price(self):
    self._setup_boundary_conditions_()
    self._setup_coefficients_()
    self._traverse_grid_()
    return self._interpolate_()







In [7]:
# Define the Finite Difference applying the explicit method: 

# The new class inherits from the FiniteDifferences class: 

class FDExplicit_EU(FiniteDifferences):
  
  def _setup_boundary_conditions_(self):
    if self.callable:# calculations if the option is a call
      self.grid[:, -1] = np.maximum(self.boundary_conds - self.K, 0)
      self.grid[-1, :-1] = (self.Smax - self.K)*np.exp(-self.r*self.dt*(self.N - self.j_values))

    else: 
      # calculation if the option is a put:
      self.grid[:, -1] = np.maximum(self.K - self.boundary_conds, 0)
      self.grid[0, :-1] = (self.K - self.Smax)*np.exp(-self.r*self.dt*(self.N - self.j_values))

  # Define a method for FDExplicit Class to calculate coefficients for the FD option pricing, the method is inherited from the FiniteDifferences Class 
  # but is defined for each FD approach separately: 
  def _setup_coefficients_(self):
    # calculate coefficient "a" for the explicit algorithm:
    self.a = 0.5*self.dt*((self.sigma**2) * (self.i_values**2) - self.r*self.i_values)
    # calculate coefficient "b" for the explicit algorithm:
    self.b = 1 - self.dt*((self.sigma**2)*(self.i_values**2) + self.r)
    # calculate coefficient "c" for the explicit algorithm:
    self.c = 0.5*self.dt*((self.sigma**2)*(self.i_values**2) + self.r*self.i_values)

  def _traverse_grid_(self):
    for j in reversed(self.j_values):
      for i in range(self.M)[2:]:
        self.grid[i,j] = self.a[i]*self.grid[i-1, j+1]+\
                         self.b[i]*self.grid[i, j+1] +\
                         self.c[i]*self.grid[i+1, j+1]

In [21]:
# Define the Finite Difference applying the implicit method:
# This method uses a system of linear equations to derive the option price: 

# define the FDImplicit_EU class which inherits from the FDExplicit_EU class: 

class FDImplicit_EU(FDExplicit_EU):

  def _setup_coefficients_(self):
    self.a = 0.5*(self.r*self.dt*self.i_values - (self.sigma**2)*self.dt*(self.i_values**2))

    self.b = 1 + (self.sigma**2)*self.dt*(self.i_values**2) + self.r*self.dt

    self.c = -0.5*(self.r*self.dt*self.i_values + (self.sigma**2)*self.dt*(self.i_values**2))

    self.coeffs = np.diag(self.a[2:self.M], -1) + np.diag(self.b[1:self.M]) + np.diag(self.c[1:self.M - 1], 1)

  def _traverse_grid_(self): 
    # solve by using linear systems of equations: 
    P, L, U = linalg.lu(self.coeffs)
    aux = np.zeros(self.M - 1)

    for j in reversed(range(self.N)):
      aux[0] = np.dot(-self.a[1], self.grid[0,j])

      x1 = linalg.solve(L, self.grid[1:self.M, j+1] + aux)
      x2 = linalg.solve(U, x1)
      self.grid[1:self.M, j] = x2


In [30]:
# Define Crank-Nicolson method: 

class FDCrank_Nicolson_EU(FDExplicit_EU):

  def _setup_coefficients_(self):
    self.a = 0.25*self.dt*((self.sigma**2)*(self.i_values**2) - self.r*self.i_values)

    self.b = -0.5*self.dt*((self.sigma**2)*(self.i_values**2) + self.r)

    self.c = 0.25*self.dt*((self.sigma**2)*(self.i_values**2) + self.r*self.i_values)

    self.M1 = -np.diag(self.a[2:self.M], -1) + np.diag(1 - self.b[1:self.M]) - np.diag(self.c[1:self.M-1], 1)

    self.M2 = np.diag(self.a[2:self.M], -1) + np.diag(1 + self.b[1:self.M]) + np.diag(self.c[1:self.M - 1], 1)


  def _traverse_grid_(self):
    P, L, U = linalg.lu(self.M1)

    for j in reversed(range(self.N)):
      x1 = linalg.solve(L, np.dot(self.M2, self.grid[1:self.M, j+1]))

      x2 = linalg.solve(U, x1)

      self.grid[1:self.M, j] = x2


# Crank Nicolson is the most prefered method for 2 reasons: 
# 1) It avoids the instability issue
# 2) It converges faster than explicit and implicit methods

In [34]:
# Define class to calculate the American style option using the Finite Difference approach with Crank Nicolson method: 

class FD_Crank_Nicolson_Am(FDCrank_Nicolson_EU):
  pass

# To be done later!

## Testing the code:

### Finite Difference option pricing with Explicit method: 

In [14]:
# Testing the Explicit Finite Difference option pricing: 

option_explicit_stable = FDExplicit_EU(50, 50, 0.1, 5./12., 0.4, 100, 100, 1000, False)
print("The option's value applying the Finite Difference pricing with the explicit method is: " + str(option_explicit_stable.price()))


The option's value applying the Finite Difference pricing with the explicit method is: 4.072882278148043


In [15]:
# However, the explicit method is not always stable and therefore, it does not always converge to a reasonable solution:
option_explicit_instable = FDExplicit_EU(50, 50, 0.1, 5./12., 0.4, 100, 100, 100, False)
print("The option's value applying the Finite Difference pricing with the explicit method is: " + str(option_explicit_instable.price()))

The option's value applying the Finite Difference pricing with the explicit method is: -1.6291077072251005e+53


### Finite Difference option pricing with Implicit method:

In [22]:
# Explicit method was not able to converge to a reasonable solution for the defined option:
option_1 = FDImplicit_EU(50, 50, 0.1, 5./12., 0.4, 100, 100, 100, False)
print("The option's value applying the implicit method is: " + str(option_1.price()))

The option's value applying the implicit method is: 4.065801939431454


In [24]:
option_2 = FDImplicit_EU(50, 50, 0.1, 5./12., 0.4, 100, 100, 1000, False)
print("The option's value applying the implicit method is: " + str(option_2.price()))

The option's value applying the implicit method is: 4.071594188049893


### Finite Difference option pricing with Crank-Nicolson method:

In [31]:
option_CN_1 = FDCrank_Nicolson_EU(50, 50, 0.1, 5./12., 0.4, 100, 100, 100, False)
print("The option's value applying the Crank Nicolson method is: " + str(option_CN_1.price()))

The option's value applying the Crank Nicolson method is: 4.072254507998114


In [32]:
option_CN_2 = FDCrank_Nicolson_EU(50, 50, 0.1, 5./12., 0.4, 100, 100, 1000, False)
print("The option's value applying the Crank Nicolson method is: " + str(option_CN_2.price()))

The option's value applying the Crank Nicolson method is: 4.072238354486825
